# **Proyecto final bootcamp programacion Python**
Demostracion de funcionamiento en el siguiente link: [https://www.youtube.com/watch?v=7A3k4RgOcHs](https://www.youtube.com/watch?v=7A3k4RgOcHs)

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
from datetime import datetime

# === Variables de datos globales ===
df_stock = pd.DataFrame({
    "Codigo": [1001, 2001, 3001, 4001, 5001],
    "Producto": ['Pantalon', 'Jean', 'Camisa', 'Blusa', 'Short'],
    "Cantidad": [10, 15, 20, 8, 5],
    "Precio": [55000, 70000, 35000, 45000, 50000]
})

df_ventas = pd.DataFrame(columns=["Fecha", "Producto", "Cantidad", "Precio Unitario","Total"])

# === Funciones ===
def ver_stock(cargar_datos=False):
    if not cargar_datos:
        return pd.DataFrame(columns=["Codigo", "Producto", "Cantidad", "Precio"]), ""

    alerta = "\n"
    stock_bajo = df_stock[df_stock["Cantidad"] <= 5]
    if not stock_bajo.empty:
        alerta += "🚨 Productos con poco stock:\n"
        for _, fila in stock_bajo.iterrows():
            alerta += f"❗ {fila['Producto']} - {fila['Cantidad']} unidades\n"
    else:
        alerta = "✅ Todos los productos tienen suficiente stock."
    return df_stock, alerta

def actualizar_dropdown():
    return gr.update(choices=df_stock["Producto"].tolist())

def obtener_productos():
    return df_stock["Producto"].tolist()

def cargar_productos_dropdown():
    return gr.update(choices=df_stock["Producto"].tolist())

def vender(producto, cantidad):
    global df_stock, df_ventas
    if producto not in df_stock["Producto"].values:
        return "❌ Producto no encontrado en stock."

    if cantidad <= 0:
        return "❌ La cantidad debe ser mayor a cero."

    fila_producto = df_stock[df_stock["Producto"] == producto].index[0]

    if df_stock.at[fila_producto, "Cantidad"] < cantidad:
        return f"❌ No hay suficiente stock. Disponibles: {df_stock.at[fila_producto, 'Cantidad']} unidades."

    # Actualizar stock
    df_stock.at[fila_producto, "Cantidad"] -= cantidad

    # Calcular precios
    precio_unitario = df_stock.at[fila_producto, "Precio"]
    subtotal = precio_unitario * cantidad
    fecha_actual = datetime.now().strftime("%d/%m/%Y")

    # Guardar en df_ventas
    df_ventas.loc[len(df_ventas)] = [fecha_actual, producto, cantidad, precio_unitario, subtotal]

    # Generar factura estilo tabla
    factura = f"""
    🧾 FACTURA DE COMPRA - {fecha_actual}
    --------------------------------------------
    Producto        : {producto}
    Cantidad        : {cantidad}
    Valor unitario  : ${precio_unitario:,.0f}
    Subtotal        : ${subtotal:,.0f}
    --------------------------------------------
    TOTAL A PAGAR   : ${subtotal:,.0f}
    """
    return factura

def reporte(cargar_datos=False):
    if not cargar_datos:
        return pd.DataFrame(columns=["Fecha", "Producto", "Cantidad", "Total"])
    return df_ventas

def grafica():
    global df_ventas
    if df_ventas.empty:
        return plt.figure()
    resumen = df_ventas.groupby("Producto")["Cantidad"].sum()
    fig, ax = plt.subplots()
    resumen.plot(kind="bar", ax=ax, color="magenta")
    ax.set_title("Productos más vendidos")
    ax.set_ylabel("Cantidad Vendida")
    ax.set_xlabel("Producto")
    return fig

def agregar_producto(codigo, producto, cantidad, precio):
    global df_stock

    # Convertir valores numéricos (manejar casos vacíos/None)
    try:
        codigo = int(codigo) if codigo not in [None, ""] else None
        cantidad = int(cantidad) if cantidad not in [None, ""] else None
        precio = float(precio) if precio not in [None, ""] else None
    except:
        raise gr.Error("❌ Código, cantidad y precio deben ser números válidos")

    # Validación de campos obligatorios
    if None in [codigo, producto, cantidad, precio] or not str(producto).strip():
        raise gr.Error("❌ Todos los campos son obligatorios")

    # Validación de valores positivos
    if any(val <= 0 for val in [codigo, cantidad, precio]):
        raise gr.Error("❌ Todos los valores numéricos deben ser positivos")

    # Validación de código único
    if (codigo in df_stock["Codigo"].values and
        not df_stock[df_stock["Codigo"] == codigo]["Producto"].str.lower().eq(producto.lower()).any()):
        raise gr.Error(f"❌ El código {codigo} ya existe con otro producto")

    # Lógica de agregar/actualizar (case-insensitive)
    producto_lower = producto.lower()
    mask = df_stock["Producto"].str.lower() == producto_lower

    if mask.any():
        # Sumar cantidad al producto existente y actualizar precio
        df_stock.loc[mask, "Cantidad"] += cantidad
        df_stock.loc[mask, "Precio"] = precio  # Actualiza el precio al nuevo valor
        return f"🔄 Producto '{producto}' actualizado (Se añadieron {cantidad} unidades)"
    else:
        # Agregar nuevo producto
        nuevo = {"Codigo": codigo, "Producto": producto, "Cantidad": cantidad, "Precio": precio}
        df_stock.loc[len(df_stock)] = nuevo
        return f"✅ Producto '{producto}' agregado"
def limpiar_campos():
    return {
        codigo: None,
        producto: "",
        cantidad: None,
        precio: None,
        salida_agregar: ""
    }

# === Interfaz visual con Blocks ===
with gr.Blocks(title="Fashion Stock Manager") as demo:
    gr.Markdown("""
    # 🌷🌹 Valeria Boutique - Sistema de Inventario 🌹🌷
    Bienvenida al sistema de control de stock y ventas.
    """)

    with gr.Tab("📦 Ver Stock"):
        # Mostrar tabla vacía inicialmente
        output_stock = gr.Dataframe(
            value=pd.DataFrame(columns=["Codigo", "Producto", "Cantidad", "Precio"]),
            headers=["Código", "Producto", "Cantidad", "Precio"],
            interactive=False
        )
        alerta_texto = gr.Textbox(label="Alertas", lines=2, value="")
        # Botón para cargar datos reales
        btn_stock = gr.Button("📦 Consultar stock disponible")
        # Configurar el evento del botón
        btn_stock.click(
            fn=lambda: ver_stock(cargar_datos=True),
            outputs=[output_stock, alerta_texto]
        )

    with gr.Tab("🛍️ Realizar Venta"):
        with gr.Blocks() as tab_vender:
            dropdown_producto = gr.Dropdown(label="Producto")
            cantidad_input = gr.Number(label="Cantidad", precision=0)
            btn_enviar = gr.Button("🛍️ Comprar")
            output_text = gr.Textbox(label="Resultado")
            # Acción cuando se da clic en "Enviar"
            btn_enviar.click(fn=vender, inputs=[dropdown_producto, cantidad_input], outputs=output_text)
            # Actualizar el dropdown al cargar
            tab_vender.load(fn=cargar_productos_dropdown, inputs=[], outputs=dropdown_producto)

    with gr.Tab("📄 Reporte de Ventas"):
        # Mostrar tabla vacía inicialmente con las mismas columnas que df_ventas
        df_ventas_output = gr.Dataframe(
            value=pd.DataFrame(columns=["Fecha", "Producto", "Cantidad", "Total"]),
            headers=["Fecha", "Producto", "Cantidad", "Total"],
            interactive=False
        )
        # Botón para cargar datos reales
        btn_reporte = gr.Button("📄 Ver reporte")
        # Configurar el evento del botón
        btn_reporte.click(
            fn=lambda: reporte(cargar_datos=True),
            outputs=df_ventas_output
        )

    with gr.Tab("📊 Gráfica"):
        btn_grafica = gr.Button("📊 Ver gráfica")
        plot_output = gr.Plot()
        btn_grafica.click(fn=grafica, outputs=plot_output)

    with gr.Tab("➕ Agregar/Actualizar Producto"):
        with gr.Row():
            codigo = gr.Number(label="Código", precision=0)
            producto = gr.Text(label="Producto")
        with gr.Row():
            cantidad = gr.Number(label="Cantidad", precision=0)
            precio = gr.Number(label="Precio", precision=0)

        btn_agregar = gr.Button("➕ Agregar/Actualizar", variant="primary")
        btn_limpiar = gr.Button("🧹 Limpiar Campos", variant="secondary")
        salida_agregar = gr.Textbox(label="Resultado")

        # Eventos
        btn_agregar.click(
            fn=agregar_producto,
            inputs=[codigo, producto, cantidad, precio],
            outputs=salida_agregar
        ).then(
            lambda: [None, "", None, None],  # Limpia campos
            outputs=[codigo, producto, cantidad, precio]
        ).then(
            cargar_productos_dropdown,
            outputs=dropdown_producto
        )

        btn_limpiar.click(
            fn=lambda: [None, "", None, None, ""],  # Limpia TODOS los campos
            outputs=[codigo, producto, cantidad, precio, salida_agregar]
        )

# Ejecutar
if __name__ == '__main__':
    demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6953a2c0d7dd606ea4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
